In [ ]:
workspace_id = 'ecb20071-df83-4940-bd22-4eff25231269'
dataset_id   = '0a9222df-e30b-4272-bbde-0b0a48349e24'

In [ ]:
import requests
import json
import msal
from time import sleep

def refresh_dataset(workspace_id, dataset_id):
    client_id = "REDACTED-GUID"
    client_secret = "REDACTED-SECRET-PHRASE"
    tenant_id = "REDACTED-GUID"

    scope = ["https://analysis.windows.net/powerbi/api/.default"]
    api_url = (
        "https://api.powerbi.com/v1.0/myorg/groups/"
        + workspace_id
        + "/datasets/"
        + dataset_id
        + "/refreshes"
    )

    # Use MSAL to grab token
    # Initialize MSAL client
    app = msal.ConfidentialClientApplication(
        client_id=client_id,
        client_credential=client_secret,
        authority=f"https://login.microsoftonline.com/{tenant_id}",
    )

    # Acquire a token for Power BI API
    result = app.acquire_token_for_client(scopes=scope)

    # ping the API using a request, given we have a valid token
    if "access_token" in result:
        access_token = result["access_token"]
        # Prepare the request headers
        headers = {
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json",
        }

        # Send a GET request to the Power BI REST API
        reauth_count = 0
        response = requests.post(api_url, headers=headers)
        # print(f"{response.status_code} - {response.text}")
        # 202 means the request has been accepted for processing, but the processing has not been completed, this is good!
        while True:
            if response.status_code == 202:
                return "Refresh Underway Id=" + str(response.headers["RequestId"])
            elif response.status_code == 400:
                sleep(15)
            elif response.status_code == 401 and reauth_count < 5:
                result = app.acquire_token_for_client(scopes=scope)
                headers["Authorization"] = f"Bearer {access_token}"
                reauth_count = reauth_count + 1
            else:
                return (
                    "Unexpected Response from API "
                    + str(response.status_code)
                    + "-"
                    + response.text
                )
            response = requests.post(api_url, headers=headers)


In [ ]:
session = requests.Session()
refresh_dataset(workspace_id, dataset_id)